In [24]:
import sklearn
from sklearn.model_selection import train_test_split
import shutil
import os
import pandas as pd

In [25]:
scrpath = "/home/user/TensorFlow/workspace/training_AS/DxAS/AS_final_jpg_200928"
train_path = "/home/user/TensorFlow/workspace/training_AS/images/train"
test_path = "/home/user/TensorFlow/workspace/training_AS/images/test"
csvpath = "/home/user/TensorFlow/workspace/training_AS/annotations/AS_annotation_2020Sep.csv"

config_path = "/home/user/TensorFlow/workspace/training_AS/models/efficientdet_d4_coco17_tpu-32/pipeline.config"
model_path = "/home/user/TensorFlow/workspace/training_AS/models/efficientdet_d4_coco17_tpu-32/"

In [47]:
os.chdir("/home/user/TensorFlow/")

for i in range(50):
    print(i)
    csvfile = pd.read_csv(csvpath)
    col_list = csvfile.columns.to_list()
    
    data = list(set(csvfile.Filename.to_list()))
    label = list(csvfile.label_patient)[::2]

    print(len(data), len(label))
    
    #split dataset
    trainX, testX, train_y, test_y = train_test_split(data, label, stratify=label, test_size=0.2, random_state=i+1)

    len(trainX), len(testX), len(train_y), len(test_y)
    print("dataset splited")
#    print(trainX)
   
    #move train_images to train folder
    total = 0
    shutil.rmtree(train_path)
    os.mkdir(train_path)
    for train in trainX:
        total += 1
        shutil.copy(os.path.join(scrpath, train), os.path.join(train_path, train))
    print(str(total)+" file moved")
    
    # train.csv
    train_df = pd.DataFrame(columns = col_list)
    for file in os.listdir(train_path):
        row = csvfile.loc[csvfile["Filename"]== file]
        train_df = pd.concat([train_df, row], axis=0, sort=False)
    train_df.to_csv(os.path.join(train_path, "train.csv"))
    print("train.csv created")
    
    #move test_images to test folder
    total = 0
    shutil.rmtree(test_path)
    os.mkdir(test_path)
    for test in testX:
        total += 1
        shutil.copy(os.path.join(scrpath, test), os.path.join(test_path, test))
    print(str(total)+" file moved")
    
    # test.csv
    test_df = pd.DataFrame(columns = col_list)
    for file in os.listdir(test_path):
        row = csvfile.loc[csvfile["Filename"]== file]
        test_df = pd.concat([test_df, row], axis=0, sort=False)
    test_df.to_csv(os.path.join(test_path, "test.csv"))
    print("test.csv created")
    
    # create tf2 .records
    !python ./scripts/preprocessing/build_ankylosing_records_tf2.py --csv_dir "/home/user/TensorFlow/workspace/training_AS/images/train/train.csv" --labels_path "/home/user/TensorFlow/workspace/training_AS/annotations/label_map.pbtxt" --output_path "/home/user/TensorFlow/workspace/training_AS/annotations/train.record" --image_dir "/home/user/TensorFlow/workspace/training_AS/images/train/"
    !python ./scripts/preprocessing/build_ankylosing_records_tf2.py --csv_dir "/home/user/TensorFlow/workspace/training_AS/images/test/test.csv" --labels_path "/home/user/TensorFlow/workspace/training_AS/annotations/label_map.pbtxt" --output_path "/home/user/TensorFlow/workspace/training_AS/annotations/test.record" --image_dir "/home/user/TensorFlow/workspace/training_AS/images/test/"
    
    model_path = "/home/user/TensorFlow/workspace/training_AS/models/efficientdet_d4_coco17_tpu-32_" + str(i+1)
    os.makedirs(model_path, exist_ok=True)
    shutil.copy(config_path, os.path.join(model_path, "pipeline.config"))
    
    # train
    !python ./workspace/training_AS/model_main_tf2.py --model_dir model_path --pipeline_config_path "/home/user/TensorFlow/workspace/training_AS/models/efficientdet_d4_coco17_tpu-32/pipeline.config"

0
946 946
dataset splited
756 file moved
train.csv created
190 file moved
test.csv created
Successfully created the TFRecord file: /home/user/TensorFlow/workspace/training_AS/annotations/train.record
Successfully created the TFRecord file: /home/user/TensorFlow/workspace/training_AS/annotations/test.record
2020-09-25 15:08:45.856739: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-09-25 15:08:47.785823: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-09-25 15:08:47.838073: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:19:00.0 name: TITAN RTX computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.65GiB deviceMemoryBandwidth: 625.94GiB/s
2020-09-25 15:08:47.839502: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 1 with properties: 
pciBusID: 00

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
I0925 15:08:49.022187 139623474935616 cross_device_ops.py:441] Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
I0925 15:08:49.025750 139623474935616 cross_device_ops.py:441] Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
I0925 15:08:49.036510 139623474935616 efficientnet_model.py:148] round_filter input=32 output=48
I0925 15:08:49.036644 139623474935616 efficientnet_model.py:148] round_filter input=16 output=24
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
I0925 15:08:

W0925 15:09:05.538339 139623474935616 deprecation.py:317] From /home/user/anaconda3/envs/tensorflow/lib/python3.8/site-packages/object_detection/inputs.py:259: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
W0925 15:09:10.234310 139613314201344 deprecation.py:317] From /home/user/anaconda3/envs/tensorflow/lib/python3.8/site-packages/object_detection/model_lib_v2.py:355: set_learning_phase (from tensorflow.python.keras.backend) is deprecated and will be removed after 2020-10-11.
Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
2020-09-25 15:09:54.303536: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10


W0925 15:10:22.840345 139613314201344 optimizer_v2.py:1273] Gradients do not exist for variables ['stack_6/block_1/expand_bn/gamma:0', 'stack_6/block_1/expand_bn/beta:0', 'stack_6/block_1/depthwise_conv2d/depthwise_kernel:0', 'stack_6/block_1/depthwise_bn/gamma:0', 'stack_6/block_1/depthwise_bn/beta:0', 'stack_6/block_1/project_bn/gamma:0', 'stack_6/block_1/project_bn/beta:0', 'top_bn/gamma:0', 'top_bn/beta:0'] when minimizing the loss.
W0925 15:10:33.855547 139613322594048 optimizer_v2.py:1273] Gradients do not exist for variables ['stack_6/block_1/expand_bn/gamma:0', 'stack_6/block_1/expand_bn/beta:0', 'stack_6/block_1/depthwise_conv2d/depthwise_kernel:0', 'stack_6/block_1/depthwise_bn/gamma:0', 'stack_6/block_1/depthwise_bn/beta:0', 'stack_6/block_1/project_bn/gamma:0', 'stack_6/block_1/project_bn/beta:0', 'top_bn/gamma:0', 'top_bn/beta:0'] when minimizing the loss.
INFO:tensorflow:batch_all_reduce: 879 all-reduces with algorithm = nccl, num_packs = 1
I0925 15:10:33.944190 139623474

INFO:tensorflow:Step 2400 per-step time 1.310s loss=0.474
I0925 15:59:41.132903 139623474935616 model_lib_v2.py:649] Step 2400 per-step time 1.310s loss=0.474
INFO:tensorflow:Step 2500 per-step time 1.223s loss=0.353
I0925 16:01:43.331157 139623474935616 model_lib_v2.py:649] Step 2500 per-step time 1.223s loss=0.353
INFO:tensorflow:Step 2600 per-step time 1.056s loss=0.464
I0925 16:03:43.079105 139623474935616 model_lib_v2.py:649] Step 2600 per-step time 1.056s loss=0.464
INFO:tensorflow:Step 2700 per-step time 1.181s loss=0.443
I0925 16:05:44.291353 139623474935616 model_lib_v2.py:649] Step 2700 per-step time 1.181s loss=0.443
INFO:tensorflow:Step 2800 per-step time 1.233s loss=0.361
I0925 16:07:44.363613 139623474935616 model_lib_v2.py:649] Step 2800 per-step time 1.233s loss=0.361
INFO:tensorflow:Step 2900 per-step time 1.230s loss=0.355
I0925 16:09:44.842694 139623474935616 model_lib_v2.py:649] Step 2900 per-step time 1.230s loss=0.355
INFO:tensorflow:Step 3000 per-step time 1.241s

...

2020-09-28 08:51:16.480623: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1858] Adding visible gpu devices: 0, 1
2020-09-28 08:51:16.480664: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-09-28 08:51:17.320925: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1257] Device interconnect StreamExecutor with strength 1 edge matrix:
2020-09-28 08:51:17.320967: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1263]      0 1 
2020-09-28 08:51:17.320975: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1276] 0:   N N 
2020-09-28 08:51:17.320981: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1276] 1:   N N 
2020-09-28 08:51:17.324897: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1402] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 22465 MB memory) -> physical GPU (device: 0, name: TITAN RTX, pci bus id: 0000:19:00.0, compute capability: 7.5)
2020-09-28 08:51:17.326683:

W0928 08:51:23.595869 140483279259456 dataset_builder.py:82] num_readers has been reduced to 1 to match input file shards.
Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_deterministic`.
W0928 08:51:23.597577 140483279259456 deprecation.py:317] From /home/user/anaconda3/envs/tensorflow/lib/python3.8/site-packages/object_detection/builders/dataset_builder.py:96: parallel_interleave (from tensorflow.python.data.experimental.ops.interleave_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_deterministic`.
Instructions for updating:
Use `tf.data.Dataset.map()
W0928 08:51:23.616399 140483279

W0928 08:52:42.853672 140473196861184 optimizer_v2.py:1273] Gradients do not exist for variables ['stack_6/block_1/expand_bn/gamma:0', 'stack_6/block_1/expand_bn/beta:0', 'stack_6/block_1/depthwise_conv2d/depthwise_kernel:0', 'stack_6/block_1/depthwise_bn/gamma:0', 'stack_6/block_1/depthwise_bn/beta:0', 'stack_6/block_1/project_bn/gamma:0', 'stack_6/block_1/project_bn/beta:0', 'top_bn/gamma:0', 'top_bn/beta:0'] when minimizing the loss.
W0928 08:52:53.713833 140473205253888 optimizer_v2.py:1273] Gradients do not exist for variables ['stack_6/block_1/expand_bn/gamma:0', 'stack_6/block_1/expand_bn/beta:0', 'stack_6/block_1/depthwise_conv2d/depthwise_kernel:0', 'stack_6/block_1/depthwise_bn/gamma:0', 'stack_6/block_1/depthwise_bn/beta:0', 'stack_6/block_1/project_bn/gamma:0', 'stack_6/block_1/project_bn/beta:0', 'top_bn/gamma:0', 'top_bn/beta:0'] when minimizing the loss.
INFO:tensorflow:batch_all_reduce: 879 all-reduces with algorithm = nccl, num_packs = 1
I0928 08:52:53.797335 140483279

INFO:tensorflow:Step 2400 per-step time 1.185s loss=0.436
I0928 09:42:15.215101 140483279259456 model_lib_v2.py:649] Step 2400 per-step time 1.185s loss=0.436
INFO:tensorflow:Step 2500 per-step time 1.178s loss=0.400
I0928 09:44:14.523481 140483279259456 model_lib_v2.py:649] Step 2500 per-step time 1.178s loss=0.400
INFO:tensorflow:Step 2600 per-step time 1.173s loss=0.421
I0928 09:46:15.032903 140483279259456 model_lib_v2.py:649] Step 2600 per-step time 1.173s loss=0.421
INFO:tensorflow:Step 2700 per-step time 1.206s loss=0.351
I0928 09:48:14.740754 140483279259456 model_lib_v2.py:649] Step 2700 per-step time 1.206s loss=0.351
INFO:tensorflow:Step 2800 per-step time 1.289s loss=0.488
I0928 09:50:15.363677 140483279259456 model_lib_v2.py:649] Step 2800 per-step time 1.289s loss=0.488
INFO:tensorflow:Step 2900 per-step time 1.160s loss=0.414
I0928 09:52:14.747981 140483279259456 model_lib_v2.py:649] Step 2900 per-step time 1.160s loss=0.414
INFO:tensorflow:Step 3000 per-step time 1.275s

KeyboardInterrupt: 

In [3]:
#export graph
import os
os.chdir("/home/user/TensorFlow/")
base_path = "/home/user/TensorFlow/workspace/training_AS/models/"

for file in os.listdir(base_path)[1:]:
    print(file)
    config_path = os.path.join(base_path, file, "pipeline.config")
    checkpoint_dir = os.path.join(base_path, file)
    output_path = "/home/user/TensorFlow/workspace/training_AS/exported-models/" + file
#    print(config_path, checkpoint_dir, output_path)
 #   os.mkdir(output_path)
    if os.listdir(output_path):
        print("alread ex")
        continue
    !python ./workspace/training_AS/exporter_main_v2.py --input_type image_tensor --pipeline_config_path {config_path} --trained_checkpoint_dir {checkpoint_dir} --output_directory {output_path}

efficientdet_d4_coco17_tpu-32_12
alread ex
efficientdet_d4_coco17_tpu-32_13
alread ex
efficientdet_d4_coco17_tpu-32_10
alread ex
efficientdet_d4_coco17_tpu-32_9
alread ex
efficientdet_d4_coco17_tpu-32_5
alread ex
efficientdet_d3_coco17_tpu-32
alread ex
efficientdet_d4_coco17_tpu-32_3
alread ex
efficientdet_d4_coco17_tpu-32_2
2020-09-29 14:40:16.984151: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-09-29 14:40:19.625238: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-09-29 14:40:19.687145: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:19:00.0 name: TITAN RTX computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.65GiB deviceMemoryBandwidth: 625.94GiB/s
2020-09-29 14:40:19.688609: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 1 with proper

I0929 14:40:24.703938 140026976290624 efficientnet_model.py:148] round_filter input=1280 output=1792
I0929 14:40:24.737889 140026976290624 efficientnet_model.py:460] Building model efficientnet with params ModelConfig(width_coefficient=1.4, depth_coefficient=1.8, resolution=380, dropout_rate=0.4, blocks=(BlockConfig(input_filters=32, output_filters=16, kernel_size=3, num_repeat=1, expand_ratio=1, strides=(1, 1), se_ratio=0.25, id_skip=True, fused_conv=False, conv_type='depthwise'), BlockConfig(input_filters=16, output_filters=24, kernel_size=3, num_repeat=2, expand_ratio=6, strides=(2, 2), se_ratio=0.25, id_skip=True, fused_conv=False, conv_type='depthwise'), BlockConfig(input_filters=24, output_filters=40, kernel_size=5, num_repeat=2, expand_ratio=6, strides=(2, 2), se_ratio=0.25, id_skip=True, fused_conv=False, conv_type='depthwise'), BlockConfig(input_filters=40, output_filters=80, kernel_size=3, num_repeat=3, expand_ratio=6, strides=(2, 2), se_ratio=0.25, id_skip=True, fused_conv=F

I0929 14:42:18.761583 140026976290624 def_function.py:1038] Unsupported signature for serialization: (([(<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f528d5d38e0>, TensorSpec(shape=(None, 128, 128, 56), dtype=tf.float32, name='feature_pyramid/0/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f528d5d37c0>, TensorSpec(shape=(None, 64, 64, 160), dtype=tf.float32, name='feature_pyramid/1/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f528d5d3e20>, TensorSpec(shape=(None, 32, 32, 448), dtype=tf.float32, name='feature_pyramid/2/1'))], False), {}).
INFO:tensorflow:Unsupported signature for serialization: (([(<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f528da3afa0>, TensorSpec(shape=(None, 128, 128, 56), dtype=tf.float32, name='feature_pyramid/0/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f528da3de50>, TensorSpec(shape=(None, 64, 64, 160), dtype=tf.float32, 

INFO:tensorflow:Unsupported signature for serialization: (([(<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f528da3afa0>, TensorSpec(shape=(None, 128, 128, 56), dtype=tf.float32, name='feature_pyramid/0/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f528da3de50>, TensorSpec(shape=(None, 64, 64, 160), dtype=tf.float32, name='feature_pyramid/1/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f528da3dbb0>, TensorSpec(shape=(None, 32, 32, 448), dtype=tf.float32, name='feature_pyramid/2/1'))], True), {}).
I0929 14:42:45.548403 140026976290624 def_function.py:1038] Unsupported signature for serialization: (([(<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f528da3afa0>, TensorSpec(shape=(None, 128, 128, 56), dtype=tf.float32, name='feature_pyramid/0/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f528da3de50>, TensorSpec(shape=(None, 64, 64, 160), dtype=tf.float32, n

2020-09-29 14:42:59.563148: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5606be5849c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2020-09-29 14:42:59.563199: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): TITAN RTX, Compute Capability 7.5
2020-09-29 14:42:59.563218: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): TITAN RTX, Compute Capability 7.5
2020-09-29 14:42:59.566157: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:19:00.0 name: TITAN RTX computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.65GiB deviceMemoryBandwidth: 625.94GiB/s
2020-09-29 14:42:59.569073: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 1 with properties: 
pciBusID: 0000:65:00.0 name: TITAN RTX computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.65GiB deviceMemo

...

INFO:tensorflow:Unsupported signature for serialization: (([(<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fcaa1f7e5b0>, TensorSpec(shape=(None, 128, 128, 56), dtype=tf.float32, name='feature_pyramid/0/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fcaa1f78d00>, TensorSpec(shape=(None, 64, 64, 160), dtype=tf.float32, name='feature_pyramid/1/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fcaa1f78eb0>, TensorSpec(shape=(None, 32, 32, 448), dtype=tf.float32, name='feature_pyramid/2/1'))], False), {}).
I0929 14:56:03.729059 140542658471744 def_function.py:1038] Unsupported signature for serialization: (([(<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fcaa1f7e5b0>, TensorSpec(shape=(None, 128, 128, 56), dtype=tf.float32, name='feature_pyramid/0/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fcaa1f78d00>, TensorSpec(shape=(None, 64, 64, 160), dtype=tf.float32, 

INFO:tensorflow:Assets written to: /home/user/TensorFlow/workspace/training_AS/exported-models/efficientdet_d4_coco17_tpu-32_6/saved_model/assets
I0929 14:56:22.588911 140542658471744 builder_impl.py:774] Assets written to: /home/user/TensorFlow/workspace/training_AS/exported-models/efficientdet_d4_coco17_tpu-32_6/saved_model/assets
INFO:tensorflow:Writing pipeline config file to /home/user/TensorFlow/workspace/training_AS/exported-models/efficientdet_d4_coco17_tpu-32_6/pipeline.config
I0929 14:56:26.177310 140542658471744 config_util.py:253] Writing pipeline config file to /home/user/TensorFlow/workspace/training_AS/exported-models/efficientdet_d4_coco17_tpu-32_6/pipeline.config
efficientdet_d4_coco17_tpu-32_7
2020-09-29 14:56:32.176523: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-09-29 14:56:33.764484: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcu

I0929 14:56:37.217019 139753815598912 efficientnet_model.py:148] round_filter input=112 output=160
I0929 14:56:37.217158 139753815598912 efficientnet_model.py:148] round_filter input=192 output=272
I0929 14:56:38.108687 139753815598912 efficientnet_model.py:148] round_filter input=192 output=272
I0929 14:56:38.108827 139753815598912 efficientnet_model.py:148] round_filter input=320 output=448
I0929 14:56:38.285347 139753815598912 efficientnet_model.py:148] round_filter input=1280 output=1792
I0929 14:56:38.319526 139753815598912 efficientnet_model.py:460] Building model efficientnet with params ModelConfig(width_coefficient=1.4, depth_coefficient=1.8, resolution=380, dropout_rate=0.4, blocks=(BlockConfig(input_filters=32, output_filters=16, kernel_size=3, num_repeat=1, expand_ratio=1, strides=(1, 1), se_ratio=0.25, id_skip=True, fused_conv=False, conv_type='depthwise'), BlockConfig(input_filters=16, output_filters=24, kernel_size=3, num_repeat=2, expand_ratio=6, strides=(2, 2), se_rati

INFO:tensorflow:Unsupported signature for serialization: (([(<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f12e73bc700>, TensorSpec(shape=(None, 128, 128, 56), dtype=tf.float32, name='feature_pyramid/0/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f12e73bc280>, TensorSpec(shape=(None, 64, 64, 160), dtype=tf.float32, name='feature_pyramid/1/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f12e73bca00>, TensorSpec(shape=(None, 32, 32, 448), dtype=tf.float32, name='feature_pyramid/2/1'))], False), {}).
I0929 14:58:42.837440 139753815598912 def_function.py:1038] Unsupported signature for serialization: (([(<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f12e73bc700>, TensorSpec(shape=(None, 128, 128, 56), dtype=tf.float32, name='feature_pyramid/0/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f12e73bc280>, TensorSpec(shape=(None, 64, 64, 160), dtype=tf.float32, 

INFO:tensorflow:Unsupported signature for serialization: (([(<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f12e73bc700>, TensorSpec(shape=(None, 128, 128, 56), dtype=tf.float32, name='feature_pyramid/0/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f12e73bc280>, TensorSpec(shape=(None, 64, 64, 160), dtype=tf.float32, name='feature_pyramid/1/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f12e73bca00>, TensorSpec(shape=(None, 32, 32, 448), dtype=tf.float32, name='feature_pyramid/2/1'))], False), {}).
I0929 14:58:58.735871 139753815598912 def_function.py:1038] Unsupported signature for serialization: (([(<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f12e73bc700>, TensorSpec(shape=(None, 128, 128, 56), dtype=tf.float32, name='feature_pyramid/0/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f12e73bc280>, TensorSpec(shape=(None, 64, 64, 160), dtype=tf.float32, 